### Cài đặt thư viện và thiết lập Logging
Chúng ta sẽ cài đặt thư viện pandas, numpy, sklearn và thiết lập logging để ghi nhận thông tin trong quá trình xử lý.



In [1]:
import os
import logging
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Thiết lập Logging
logging.basicConfig(format="%(asctime)s %(levelname)s %(message)s", datefmt="%H:%M:%S", level=logging.INFO)

# Cài đặt hiển thị cho DataFrame (hiển thị tối đa 85 hàng)
pd.set_option("display.max_rows", 85)


### Đặt đường dẫn tới tệp dữ liệu
Xác định đường dẫn tới thư mục chứa dữ liệu `MachineLearningCVE.csv` để xử lý.


In [7]:
# Cập nhật đường dẫn
DIR_PATH = "MachineLearningCSV/MachineLearningCVE"
PROCESSED_DIR_PATH = "ProcessedDataset"
FILE_PATH = os.path.join(DIR_PATH, "MachineLearningCVE.csv")


### Mã hóa nhãn `Label` bằng Label Encoding
Sử dụng `LabelEncoder` để chuyển đổi cột `Label` từ dạng chuỗi sang số nhằm chuẩn bị cho việc huấn luyện mô hình. Kết quả được lưu trữ để sử dụng trong các bước tiếp theo.


In [8]:
def _label_encoding() -> LabelEncoder:
    # Khởi tạo Label Encoder
    le = LabelEncoder()

    # Đọc cột `Label` từ tệp CSV
    labels = pd.read_csv(FILE_PATH, usecols=['Label'], skipinitialspace=True)

    # Huấn luyện bộ mã hóa với dữ liệu từ `Label`
    le.fit(labels.Label)

    # Lưu bộ mã hóa
    np.save(os.path.join(PROCESSED_DIR_PATH, 'label_encoder.npy'), le.classes_)

    # Log kết quả
    logging.info("Total rows: {}".format(labels.shape))
    logging.info("Class distribution:\n{}\n".format(labels.Label.value_counts()))

    return le

label_encoder = _label_encoding()


15:13:25 INFO Total rows: (2830743, 1)
15:13:25 INFO Class distribution:
Label
BENIGN                        2273097
DoS Hulk                       231073
PortScan                       158930
DDoS                           128027
DoS GoldenEye                   10293
FTP-Patator                      7938
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1966
Web Attack � Brute Force         1507
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Heartbleed                         11
Name: count, dtype: int64



### Xử lý dữ liệu
Thực hiện các bước xử lý dữ liệu để thay thế các giá trị `NaN`, giá trị vô hạn (`inf`) và giá trị âm bằng các giá trị phù hợp dựa trên lớp của từng hàng.


In [10]:
def _process(df: pd.DataFrame, le: LabelEncoder) -> pd.DataFrame:
    # Mã hóa cột Label
    df.Label = le.transform(df.Label)

    # Điền giá trị NaN với giá trị trung bình của từng lớp
    nan_rows = df[df.isna().any(axis=1)].shape[0]
    logging.info("Fill NaN in {} rows with average value of each class.".format(nan_rows))
    df.iloc[:, df.columns != "Label"] = df.groupby("Label").transform(lambda x: x.fillna(x.mean()))

    # Thay thế giá trị vô hạn với giá trị tối đa của từng lớp
    inf_rows = df[df.isin([np.inf]).any(axis=1)].shape[0]
    logging.info("Replace Inf in {} rows with maximum value of each class.".format(inf_rows))
    df = df.replace([np.inf], np.nan)
    df.iloc[:, df.columns != "Label"] = df.groupby("Label").transform(lambda x: x.fillna(x.max()))

    # Thay thế giá trị âm với giá trị dương nhỏ nhất của từng lớp
    logging.info("Replace negative values with minimum value of each class.")
    df[df < 0] = np.nan
    df.iloc[:, df.columns != "Label"] = df.groupby("Label").transform(lambda x: x.fillna(x.min()))

    return df


### Chia dữ liệu thành tập huấn luyện và kiểm tra
Tách dữ liệu thành tập huấn luyện và tập kiểm tra với tỷ lệ 80:20 để chuẩn bị cho quá trình huấn luyện và đánh giá mô hình.


In [11]:
def _split_train_test(df: pd.DataFrame) -> (pd.DataFrame, pd.DataFrame):
    # Tách dữ liệu thành X và y
    x = df.iloc[:, df.columns != 'Label']
    y = df[['Label']]

    # Chia thành tập huấn luyện và kiểm tra
    x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, test_size=0.20, random_state=np.random.randint(10))

    del x, y

    train = pd.concat([x_train, y_train], axis=1, sort=False)
    test = pd.concat([x_test, y_test], axis=1, sort=False)

    return train, test


### Lưu dữ liệu đã xử lý vào tệp CSV
Lưu tập dữ liệu đã xử lý vào tệp CSV để có thể sử dụng lại mà không cần xử lý lại từ đầu.


In [12]:
def _to_csv(df: pd.DataFrame, saving_path: str):
    # Nếu file chưa tồn tại, ghi tiêu đề và dữ liệu
    if not os.path.isfile(saving_path):
        df.to_csv(saving_path, index=False)
    # Nếu file đã tồn tại, chỉ ghi dữ liệu mà không ghi tiêu đề
    else:
        df.to_csv(saving_path, index=False, mode='a', header=False)


### Tiền xử lý dữ liệu theo từng khối
Chia dữ liệu thành từng khối nhỏ để xử lý tuần tự, giúp tránh quá tải bộ nhớ khi làm việc với tập dữ liệu lớn.


In [13]:
def _preprocessing_all(le: LabelEncoder, chunksize=1000000):
    # Tiền xử lý toàn bộ dữ liệu từ tệp CSV
    for chunk in pd.read_csv(FILE_PATH, skipinitialspace=True, chunksize=chunksize):
        train, test = _split_train_test(_process(chunk, le))
        _to_csv(train, os.path.join(PROCESSED_DIR_PATH, "train_MachineLearningCVE.csv"))
        _to_csv(test, os.path.join(PROCESSED_DIR_PATH, "test_MachineLearningCVE.csv"))

_preprocessing_all(label_encoder, 2500000)


15:27:36 INFO Fill NaN in 1347 rows with average value of each class.
15:28:07 INFO Replace Inf in 2682 rows with maximum value of each class.
15:28:41 INFO Replace negative values with minimum value of each class.
15:31:04 INFO Fill NaN in 11 rows with average value of each class.
15:31:07 INFO Replace Inf in 185 rows with maximum value of each class.
15:31:08 INFO Replace negative values with minimum value of each class.
